# Apple Mobility Trend Analysis
### **Author**: Nishanth Shyam

**DISCLAIMER**: Due to the data changing daily, some of the text (majority written on 28th April 2020) will appear out of sync with the graph and underlying data and will not necessarily reflect the daily changes to the data. This was started as a personal project combining the COVID-19 data and the Apple Mobility Trends data.

In this notebook, I will be using the data available from: https://www.apple.com/covid19/mobility. I'll be focusing on UK cities but will also compare other major countries and cities to see if there are any trends which are common amongst them.

In [1]:
# Install packages
import pandas
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
pyo.init_notebook_mode(connected=True)

# Read the data file
url = 'https://raw.githubusercontent.com/nshyam97/Apple-Mobility-Trends-Data/master/applemobility.csv'
mobility_data = pandas.read_csv(url, error_bad_lines=False)

Due to the vast amount of data that is provided by Apple, I needed to reduce the amount of data we had to fit within the GitHub constraints. For that reason, the data I have filtered are all UK sub-regions and all countries that are available. All other country sub-regions are not present in this data but is present from the data available directly from Apple.

In [2]:
mobility_data.head()

,region,transportation_type,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,2020-01-19,2020-01-20,...,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22,2020-08-23
0,Albania,driving,100.0,95.30,101.43,97.20,103.55,112.67,104.83,94.39,...,248.51,267.54,255.43,229.61,216.43,209.82,228.91,243.15,244.20,248.13
1,Albania,walking,100.0,100.68,98.93,98.46,100.85,100.13,82.13,95.65,...,165.29,181.54,143.41,165.42,162.47,150.15,166.82,164.22,159.15,146.01
2,Argentina,driving,100.0,97.07,102.45,111.21,118.45,124.01,95.44,95.13,...,71.67,71.08,43.32,42.59,53.91,56.11,58.64,69.14,67.22,39.80
3,Argentina,walking,100.0,95.11,101.37,112.67,116.72,114.14,84.54,101.37,...,55.35,52.62,33.36,36.19,44.18,42.19,44.98,53.84,51.45,32.20
4,Australia,driving,100.0,102.98,104.21,108.63,109.08,89.00,99.35,103.53,...,93.84,79.57,81.30,83.35,85.40,87.96,93.81,95.41,79.51,81.64


As we can see, we have a wide table with all the dates as separate columns. While this was probably the best way to show this data without having region and transportation type repeated multiple times, it doesn't make plotting this data or filtering through this data very useful. We will need to isolate the region and the transportation type and then transpose the date columns to make it possible to plot.

Firstly, we will check what regions and transportation types we have available.

In [3]:
mobility_data['transportation_type'].unique()

array(['driving', 'walking', 'transit'], dtype=object)

In [4]:
mobility_data['region'].unique()[:200]

array(['Albania', 'Argentina', 'Australia', 'Austria', 'Belgium',
       'Brazil', 'Bulgaria', 'Cambodia', 'Canada', 'Chile', 'Colombia',
       'Croatia', 'Czech Republic', 'Denmark', 'Egypt', 'Estonia',
       'Finland', 'France', 'Germany', 'Greece', 'Hong Kong', 'Hungary',
       'Iceland', 'India', 'Indonesia', 'Ireland', 'Israel', 'Italy',
       'Japan', 'Latvia', 'Lithuania', 'Luxembourg', 'Macao', 'Malaysia',
       'Mexico', 'Morocco', 'Netherlands', 'New Zealand', 'Norway',
       'Philippines', 'Poland', 'Portugal', 'Republic of Korea',
       'Romania', 'Russia', 'Saudi Arabia', 'Serbia', 'Singapore',
       'Slovakia', 'Slovenia', 'South Africa', 'Spain', 'Sweden',
       'Switzerland', 'Taiwan', 'Thailand', 'Turkey', 'Ukraine',
       'United Arab Emirates', 'United Kingdom', 'United States',
       'Uruguay', 'Vietnam', 'Belfast', 'Birmingham', 'Bradford',
       'Bristol', 'Cardiff', 'Edinburgh', 'Glasgow', 'Leeds', 'Liverpool',
       'London', 'Manchester', 'Newcastl

In [5]:
mobility_data['region'].nunique()

86

We have 3 different transportation types but as we can see from the head of the table, some places do not have transit data. We can see the regions are the countries available from the Apple data and just the UK sub-regions as mentioned above.

In [6]:
mobility_data[mobility_data['transportation_type'] == 'transit'].shape

(43, 226)

We have 43 out of a total 86 unique regions with transit data and they are a mix of countries and cities.

Now that we know what kind of data we are working with, we can start plotting and comparing different cities and countries using various transportation types. We will start with the UK cities which I have identified from this dataset. They are: London, Leeds, Birmingham, Manchester, and Newcastle upon Tyne.

Firstly, I will create a function that will take the city or country and the transportation type and create a separate data frame with the transposed date columns as separate rows and the corresponding values. This will allow us to plot them effectively.

In [7]:
# Function to transpose the data. Currently the date values are columns and so we need to make these rows
# to make plotting the data easier
def transpose_df(region_name, transportation):
    # First choose the region and transportation type to create the usable dataframe
    df = mobility_data[(mobility_data['region'] == region_name) &
                       (mobility_data['transportation_type'] == transportation)]
    # Pivots the dataframe from a wide to a tall format. Move the Date and Values as separate rows and corresponding
    # columns.
    df_t = df.melt(['region', 'transportation_type'], var_name='Date', value_name='Value')
    # Convert date column to datetime column
    df_t.Date = pandas.to_datetime(df_t.Date, format='%Y-%m-%d')
    # Make date column the index column to allow for easier plotting
    df_t.set_index('Date', inplace=True)
    # Values are currently percentages with the first value being the baseline. To make it a change in
    # baseline mobility, minus all values by the first value.
    df_t.Value = df_t.Value - df_t.Value.iloc[0]
    # Round all the values to 2 decimal places
    df_t.Value = df_t.Value.round(2)
    # Return the finished dataframe, ready to plot
    return df_t

Now that we have this function, we can use this to create data frames for each of our UK cities so we can compare them graphically. We will be using the 'walking' transportation type initially to compare them all.

In [8]:
london = transpose_df('London', 'walking')
manchester = transpose_df('Manchester', 'walking')
birmingham = transpose_df('Birmingham', 'walking')
leeds = transpose_df('Leeds', 'walking')
newcastle = transpose_df('Newcastle upon Tyne', 'walking')

In [9]:
london.head()

,region,transportation_type,Value
Date,,,
2020-01-13,London,walking,0.00
2020-01-14,London,walking,8.89
2020-01-15,London,walking,16.84
2020-01-16,London,walking,18.82
2020-01-17,London,walking,32.18


As we can see from the London data frame example above, we now have a single data column with all the dates as rows and their corresponding values in the Value column. We also keep the region and transportation type columns which causes a lot of duplication but I felt it was better to keep them in there in case someone were to revisit this data frame later on in the code and wanted to know what it was showing. These columns can just as easily be removed as well.

Before we plot this data it is important to define what the values actually mean and why we subtracted every value by the first value. From the Apple Mobility Trends Report website, noted above, we see that the value corresponds to the "relative amount of directions requests per country/region or city compared to a baseline volume on January 13th 2020". If we look back at the initial view of the data when we first loaded it in, the baseline started at 100 and so by subtracting 100 from every value, we can display the graph as the percentage change from the baseline.

Now we can plot this data.

In [10]:
data = [london, manchester, birmingham, leeds, newcastle]
fig=go.Figure()
# Iterate through every dataframe passed through this function
for frame in data:
    # Plot the date against the corresponding value    
    fig.add_trace(go.Scatter(x=frame.index, y=frame['Value'], mode='lines', name=frame['region'][0]))

fig.update_layout(xaxis_title='Date', yaxis_title='Percentage change in direction requests (%)', 
                  title='Mobility Trend Data for ' + data[0]['transportation_type'][0])
fig.add_shape(dict(type = 'line', y0 = 0, y1 = 1, yref = 'paper',
            x0 = pandas.Timestamp('2020-03-23'), x1 = pandas.Timestamp('2020-03-23'), 
            line=dict(color='red', width=2)))
fig.add_shape(dict(type = 'line', y0 = 0, y1 = 0,
            x0 = 0, x1 = 1, xref='paper', 
            line=dict(color='black', width=2, dash='dot')))
fig.add_annotation(x=pandas.Timestamp('2020-04-26'), y=5, xref='x', yref='y', text='Baseline', showarrow=False)
fig.add_annotation(x=pandas.Timestamp('2020-03-24'), y=-30, xref='x', yref='y', 
                   text='Lockdown announced', showarrow=True, ax=65, 
                  ay=-40, arrowhead=2, arrowsize=1)
fig.show(renderer='notebook_connected')

On the graph, I have identified the date that the UK went into lockdown which was March 23rd 2020. In the run up to the lockdown, we can see that there is a pattern of increases and decreases in direction requests amongst all cities before the significant dip just before March 15th 2020.

If we look further into the data we can identify which days saw the sharp increases and which days showed the subsequent decreases. We can also look further into the abnormal dip seen between February 1st 2020 and February 15th 2020 which saw the direction requests dip further than normal.

Firstly, we will focus on the cyclical pattern before the significant change. We will look specifically at London and Birmingham as the 2 largest cities of the 4 available in the dataset.

In [11]:
london[london['Value'] > 50].sort_index()

,region,transportation_type,Value
Date,,,
2020-01-18,London,walking,60.29
2020-01-25,London,walking,62.41
2020-02-01,London,walking,61.17
2020-02-08,London,walking,67.80
2020-02-14,London,walking,50.77
2020-02-21,London,walking,53.95
2020-02-22,London,walking,70.22
2020-02-29,London,walking,53.92
2020-03-07,London,walking,67.10


In [12]:
birmingham[birmingham['Value'] > 50].sort_index()

,region,transportation_type,Value
Date,,,
2020-01-18,Birmingham,walking,61.21
2020-01-25,Birmingham,walking,92.10
2020-02-01,Birmingham,walking,82.14
2020-02-08,Birmingham,walking,66.45
2020-02-14,Birmingham,walking,65.71
2020-02-15,Birmingham,walking,60.79
2020-02-22,Birmingham,walking,73.57
2020-02-29,Birmingham,walking,89.70
2020-03-07,Birmingham,walking,95.42


To find the days which we have spikes, I've taken a 50% increase as the threshold to look at. We can see that most of the dates for both cities line up and looking at the calendar we can see these dates are all Saturday's. This would make sense as it is the start of the weekend and people and families are most likely to be walking to new places and so will need directions. I would expect the days with the dips to be on a Monday as everyone returns to work but we can confirm this by looking at all the dates before the significant dip at around March 15th 2020 and percentages less than 0.

In [13]:
london[(london['Value'] <= 0) & (london.index < pandas.Timestamp('2020-03-15'))]

,region,transportation_type,Value
Date,,,
2020-01-13,London,walking,0.00
2020-01-26,London,walking,-2.84
2020-02-09,London,walking,-23.21
2020-02-16,London,walking,-3.97
2020-03-08,London,walking,-3.14
2020-03-09,London,walking,-2.50
2020-03-12,London,walking,-1.13


In [14]:
birmingham[(birmingham['Value'] <= 0) & (birmingham.index < pandas.Timestamp('2020-03-15'))]

,region,transportation_type,Value
Date,,,
2020-01-13,Birmingham,walking,0.00
2020-01-19,Birmingham,walking,-1.68
2020-01-26,Birmingham,walking,-5.85
2020-02-09,Birmingham,walking,-23.77
2020-02-16,Birmingham,walking,-3.02
2020-02-23,Birmingham,walking,-4.57
2020-03-08,Birmingham,walking,-6.96


As we can see again, the dates generally line up with each other and they are all Sunday's which makes sense as people get ready for the working week ahead. We can also see that big dip happened on February 9th 2020 for both cities. 

Looking at headlines and major events that happened on the 9th February, we found that Storm Ciara had reached a peak and the Environment Agency had issued 186 flood alerts, 251 flood warnings, and 1 severe flood warning. Train companies had also issued "do not travel" warnings as well as a number of flights and ferries being cancelled. This is the most likely reason to have caused the significant dip as people were urged to stay at home due to the conditions of the storm. You can read more about Storm Ciara on this day [here](https://www.independent.co.uk/travel/news-and-advice/storm-ciara-travel-flights-trains-cancelled-uk-weather-latest-british-airways-easyjet-a9325456.html).

We can now look closer at the major dip on March 15th 2020. We want to look at the exact date that the dip happened so that we can see what was happening in the news during this time to see what could've caused this. We confirm that March 15th 2020 is the date we need to look at below.

In [15]:
london[(london.index >= pandas.Timestamp('2020-03-14')) & (london.index <= pandas.Timestamp('2020-03-16'))]

,region,transportation_type,Value
Date,,,
2020-03-14,London,walking,17.44
2020-03-15,London,walking,-35.72
2020-03-16,London,walking,-35.47


In [16]:
birmingham[(birmingham.index >= pandas.Timestamp('2020-03-14')) & (birmingham.index <= pandas.Timestamp('2020-03-16'))]

,region,transportation_type,Value
Date,,,
2020-03-14,Birmingham,walking,34.46
2020-03-15,Birmingham,walking,-28.91
2020-03-16,Birmingham,walking,-23.71


Looking at news sources on March 15th, we can see that the UK was hit again by another storm, Storm Dennis. You can read more about Storm Dennis on this day [here](https://www.independent.co.uk/travel/news-and-advice/storm-dennis-easyjet-flight-cancellations-half-term-gatwick-airport-heathrow-british-airways-a9337211.html). The severity of this Storm caused the Met Office to issue multiple severe weather warnings, requiring people to stay at home where possible.

While this could be one of the reasons, the main news that happened on this day was the Secretary of State for Health and Social Care, Matt Hancock warned that Britons over the age of 70 will be told to stay at home in the coming weeks. Boris Johnson, then advised on Monday morning (March 16th) that non-essential travel should be avoided as well as contact with others. He suggested that people avoid pubs, clubs and restaurants and that pregnant women and over 70s should self isolate. [source](https://www.bbc.co.uk/news/uk-51895873)

On March 12th, the government had also advised that any people with symptoms such as a continuous cough and fever should self isolate for 7 days.

Knowing this, we can use COVID-19 related data to see how this compares to the mobility trend data that we have available to us.

I am going to use data from the John Hopkins github repository found [here](https://github.com/CSSEGISandData/COVID-19). I will be focusing on the UK deaths compared with the direction requests shown above.

In [17]:
url='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'

corona_data=pandas.read_csv(url, error_bad_lines=False)
corona_data.head()
    

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1548,1554,1554,1556,1558,1574,1577,1581,1591,1595
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,536,543,549,557,559,571,579,590,598,605
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1999,2011,2024,2036,2048,2062,2077,2093,2111,2124
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,75,75,75,75,75,75,75,75,75,75
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,296,299,300,303,307,308,308,312,315,317


As we can see, the format of this data is very similar to the format that we had before with the dates all set as columns and the countries and regions as separate rows. This means that we could use a modified version of the transpose_df function we created above to convert this table into a more usable format. First, let's check if the UK is present in this dataset.

In [18]:
corona_data[corona_data['Country/Region'] == 'United Kingdom']

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20
248,Anguilla,United Kingdom,18.220600,-63.068600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249,Bermuda,United Kingdom,32.307800,-64.750500,0,0,0,0,0,0,...,9,9,9,9,9,9,9,9,9,9
250,British Virgin Islands,United Kingdom,18.420700,-64.640000,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
251,Cayman Islands,United Kingdom,19.313300,-81.254600,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
252,Channel Islands,United Kingdom,49.372300,-2.364400,0,0,0,0,0,0,...,48,48,48,48,48,48,48,48,48,48
253,Falkland Islands (Malvinas),United Kingdom,-51.796300,-59.523600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
254,Gibraltar,United Kingdom,36.140800,-5.353600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
255,Isle of Man,United Kingdom,54.236100,-4.548100,0,0,0,0,0,0,...,24,24,24,24,24,25,25,25,25,25
256,Montserrat,United Kingdom,16.742498,-62.187366,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
257,Turks and Caicos Islands,United Kingdom,21.694000,-71.797900,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6


So we have the United Kingdom present but we also have a number of overseas territories as well, divided using the Province/State column. We want to focus on mainland UK and so we will just target the row with an NaN value in the Province/State column.

In [19]:
uk_row = corona_data[(corona_data['Country/Region'] == 'United Kingdom') & (corona_data['Province/State'].isna())]
uk_row

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20
258,NaN,United Kingdom,55.3781,-3.436,0,0,0,0,0,0,...,47742,48120,48475,48888,49044,49238,49770,50365,50928,51304


Now that we have the row that we need, we can use a modified function from the one I have created earlier to get the data into a usable format for plotting. One thing also to note is that the dates are in mm/dd/yy format as opposed to the YYYY-mm-dd format used above. We will need to fix this as well.

In [20]:
# Function to transpose the data. Currently the date values are columns and so we need to make these rows
# to make plotting the data easier
def transpose_corona_row(df):
    # Drop the province/state, Lat, and Long columns as they aren't useful anymore
    df = df.drop(['Province/State', 'Lat', 'Long'], axis=1)
    # Pivots the dataframe from a wide to a tall format. Move the Date and Values as separate rows and corresponding
    # columns.
    df_t = df.melt(['Country/Region'], var_name='Date', value_name='Value')
    # Convert date column to datetime column
    df_t.Date = pandas.to_datetime(df_t.Date)
    # Make date column the index column to allow for easier plotting
    df_t.set_index('Date', inplace=True)
    # Return the finished dataframe, ready to plot
    return df_t

In [21]:
UK_deaths = transpose_corona_row(uk_row)
UK_deaths.head()

,Country/Region,Value
Date,,
2020-01-22,United Kingdom,0
2020-01-23,United Kingdom,0
2020-01-24,United Kingdom,0
2020-01-25,United Kingdom,0
2020-01-26,United Kingdom,0


Now that we have the data in a format that we are happy with, we can see that the data for corona virus dates are different to the dates provided by Apple in the mobility report. This is fine as we saw that the mobility data established a baseline before showing the big drop. That being said we can still plot both onto the same graph and see how things change as the number of deaths increases within the UK.

Instead of plotting each individual city, I will be plotting the UK as a whole.

In [22]:
UK_walking = transpose_df('United Kingdom', 'walking')
UK_driving = transpose_df('United Kingdom', 'driving')
UK_transit = transpose_df('United Kingdom', 'transit')
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=UK_walking.index, y=UK_walking.Value, mode='lines', name='Walking'), secondary_y=False)
fig.add_trace(go.Scatter(x=UK_driving.index, y=UK_driving.Value, mode='lines', name='Driving'), secondary_y=False)
fig.add_trace(go.Scatter(x=UK_transit.index, y=UK_transit.Value, mode='lines', name='Transit'), secondary_y=False)

fig.add_trace(go.Scatter(x=UK_deaths.index, y=UK_deaths.Value, mode='lines', name='Deaths'), secondary_y=True)

fig.update_layout(xaxis_title='Date', 
                  title='Mobility Trend data compared to cumulative COVID-19 deaths in the UK')
fig.update_layout(shapes=[dict(type = 'line', y0 = 0, y1 = 1, yref = 'paper',
            x0 = pandas.Timestamp('2020-03-23'), x1 = pandas.Timestamp('2020-03-23'), 
            line=dict(color='red', width=2)), 
            dict(type = 'line', y0 = 0, y1=0,
            x0 = 0, x1 = 0.94, xref='paper', 
            line=dict(color='black', width=2, dash='dot'))])
fig.add_annotation(x=pandas.Timestamp('2020-04-26'), y=5, xref='x', yref='y', text='Baseline Direction Requests', 
                   showarrow=False)
fig.add_annotation(x=pandas.Timestamp('2020-03-24'), y=-30, xref='x', yref='y', 
                   text='Lockdown announced', showarrow=True, ax=65, 
                  ay=-40, arrowhead=2, arrowsize=1)
fig.update_yaxes(title_text='Percentage change in direction requests (%)', secondary_y=False)
fig.update_yaxes(title_text='Confirmed Deaths (cumulative)', secondary_y=True)
fig.show(renderer='notebook_connected')

We can see that the deaths in the UK were just starting to rise when the lockdown was put into force and then shortly after, the number of deaths increased exponentially. As you can see, we have more up-to-date data from the John Hopkins dataset than we do from the Apple mobility trends dataset but we can see that the reduction in mobility requests decreased dramatically before the lockdown but this did not have the effect that many government officials wanted. We have only seen the deaths and we know now that the incubation period for this virus varies between 1 and 14 days before severe symptoms start to show and possibly longer before death. A more interesting metric to compare mobility data with would be to use confirmed cases of the virus.

We will use the same source as before which is taken from the John Hopkins Github page linked above. The code used to clean and transpose the death data will be the exact same as the data is set up in a similar way.

In [23]:
url='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'

corona_cases=pandas.read_csv(url, error_bad_lines=False)
corona_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,41814,41935,41975,42033,42092,42297,42463,42609,42795,42969
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,22300,22721,23210,23705,24206,24731,25294,25801,26211,26701
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,59527,60169,60800,61381,62051,62693,63446,64257,65108,65975
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,5045,5135,5135,5319,5383,5437,5477,5567,5616,5725
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,11813,12102,12223,12335,12433,12680,12816,12953,13053,13228


In [24]:
uk_cases = corona_cases[(corona_cases['Country/Region'] == 'United Kingdom') & (corona_cases['Province/State'].isna())]
uk_cases

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20
258,NaN,United Kingdom,55.3781,-3.436,0,0,0,0,0,0,...,1099059,1123197,1146484,1171441,1192013,1213363,1233775,1256725,1290195,1317496


In [25]:
UK_cases = transpose_corona_row(uk_cases)
UK_cases.head()

,Country/Region,Value
Date,,
2020-01-22,United Kingdom,0
2020-01-23,United Kingdom,0
2020-01-24,United Kingdom,0
2020-01-25,United Kingdom,0
2020-01-26,United Kingdom,0


In [26]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=UK_walking.index, y=UK_walking.Value, mode='lines', name='Walking'), secondary_y=False)
fig.add_trace(go.Scatter(x=UK_driving.index, y=UK_driving.Value, mode='lines', name='Driving'), secondary_y=False)
fig.add_trace(go.Scatter(x=UK_transit.index, y=UK_transit.Value, mode='lines', name='Transit'), secondary_y=False)

fig.add_trace(go.Scatter(x=UK_cases.index, y=UK_cases.Value, mode='lines', name='Cases'), secondary_y=True)

fig.update_layout(xaxis_title='Date', 
                  title='Mobility Trend data compared to cumulative COVID-19 cases in the UK')
fig.update_layout(shapes=[dict(type = 'line', y0 = 0, y1 = 1, yref = 'paper',
            x0 = pandas.Timestamp('2020-03-23'), x1 = pandas.Timestamp('2020-03-23'), 
            line=dict(color='red', width=2)), 
            dict(type = 'line', y0 = 0, y1=0,
            x0 = 0, x1 = 0.94, xref='paper', 
            line=dict(color='black', width=2, dash='dot'))])
fig.add_annotation(x=pandas.Timestamp('2020-04-26'), y=5, xref='x', yref='y', text='Baseline Direction Requests', 
                   showarrow=False)
fig.add_annotation(x=pandas.Timestamp('2020-03-24'), y=-30, xref='x', yref='y', 
                   text='Lockdown announced', showarrow=True, ax=65, 
                  ay=-40, arrowhead=2, arrowsize=1)
fig.update_yaxes(title_text='Percentage change in direction requests (%)', secondary_y=False)
fig.update_yaxes(title_text='Confirmed Cases (cumulative)', secondary_y=True)
fig.show(renderer='notebook_connected')

As we can see, the confirmed cases follow a very similar trend to the number of deaths. However, if we look closer, the scale is very different to the one present in the deaths, where that scale topped out at just 20,000. If we change the scale of the confirmed cases to the same one as the deaths, we can compare them more accurately.

In [27]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=UK_walking.index, y=UK_walking.Value, mode='lines', name='Walking'), secondary_y=False)
fig.add_trace(go.Scatter(x=UK_driving.index, y=UK_driving.Value, mode='lines', name='Driving'), secondary_y=False)
fig.add_trace(go.Scatter(x=UK_transit.index, y=UK_transit.Value, mode='lines', name='Transit'), secondary_y=False)

fig.add_trace(go.Scatter(x=UK_cases.index, y=UK_cases.Value, mode='lines', name='Cases'), secondary_y=True)

fig.update_layout(xaxis_title='Date', 
                  title='Mobility Trend data compared to cumulative COVID-19 cases in the UK with adjusted scale')
fig.update_layout(shapes=[dict(type = 'line', y0 = 0, y1 = 1, yref = 'paper',
            x0 = pandas.Timestamp('2020-03-23'), x1 = pandas.Timestamp('2020-03-23'), 
            line=dict(color='red', width=2)), 
            dict(type = 'line', y0 = 0, y1=0,
            x0 = 0, x1 = 0.94, xref='paper', 
            line=dict(color='black', width=2, dash='dot'))])
fig.add_annotation(x=pandas.Timestamp('2020-04-26'), y=5, xref='x', yref='y', text='Baseline Direction Requests', 
                   showarrow=False)
fig.add_annotation(x=pandas.Timestamp('2020-03-24'), y=-30, xref='x', yref='y', 
                   text='Lockdown announced', showarrow=True, ax=65, 
                  ay=-40, arrowhead=2, arrowsize=1)
fig.update_yaxes(title_text='Percentage change in direction requests (%)', secondary_y=False)
fig.update_yaxes(title_text='Confirmed Cases (cumulative)', secondary_y=True, range=[0, 20000])
fig.show(renderer='notebook_connected')

Once we convert the y-axis scale to the same scale as the deaths, we can clearly see that the exponential rise in cases started about a week before the lockdown was announced. We hit around 7500 confirmed cases at the time of the lockdown announcement and the cases continued to rise at a rapid rate even after that. Even looking at the previous graph, the cases seem to continue to rise even to this date.

However, we are seeing the effects of the lockdown with Boris Johnson claiming that the UK is "turning the tide" [source](https://news.sky.com/story/coronavirus-boris-johnson-says-easing-lockdown-would-throw-away-effort-and-sacrifice-of-british-people-11979338) and a reduction in the daily deaths starting to show at time of writing (April 28th 2020).

In [28]:
UK_deaths_diff = UK_deaths.drop('Country/Region', axis=1) #Remove the country column
UK_deaths_diff = UK_deaths_diff.diff(axis=0) #Get the difference between each row to see daily totals
UK_deaths_diff.head()

,Value
Date,
2020-01-22,NaN
2020-01-23,0.0
2020-01-24,0.0
2020-01-25,0.0
2020-01-26,0.0


In [29]:
from statsmodels.tsa.seasonal import STL

UK_deaths_diff = UK_deaths_diff.iloc[1:]

seasonal = STL(UK_deaths_diff['Value'], period=7, seasonal=7).fit().trend

fig = go.Figure()

fig.add_trace(go.Bar(x=UK_deaths_diff.index, y=UK_deaths_diff.Value, name='Deaths'))
fig.add_trace(go.Scatter(x=UK_deaths_diff.index, y=seasonal, mode='lines', name='Moving Average'))
fig.add_shape(dict(type = 'line', y0 = 0, y1 = 1, yref = 'paper',
            x0 = pandas.Timestamp('2020-03-23'), x1 = pandas.Timestamp('2020-03-23'), 
            line=dict(color='black', width=2)))
fig.update_layout(xaxis_title='Date', yaxis_title='Daily Deaths', 
                  title='Daily COVID-19 deaths in the UK')
fig.add_annotation(x=pandas.Timestamp('2020-03-22'), y=820, xref='x', yref='y', 
                   text='Lockdown announced', showarrow=True, ax=-60, 
                  ay=-45, arrowhead=2, arrowsize=1)
fig.add_annotation(x=pandas.Timestamp('2020-04-22'), y=835, xref='x', yref='y', 
                   text='7 day moving average', showarrow=True, ax=70, 
                  ay=-65, arrowhead=2, arrowsize=1, arrowcolor='red', arrowwidth=2,
                  font=dict(color='red'))
fig.update_layout(showlegend=False)
fig.show(renderer='notebook_connected')

The above graph shows the daily deaths in the UK and we can see that after the lockdown was announced, there was an exponential increase in the number of deaths. We can see however that the daily deaths are trending down which is good news and shows that the UK is slowly coming out from the peak of the virus.

With the UK being one of the most affected countries in the world by the virus, it would be interesting to see how other countries coped with the rapid spread of this virus. Many people are praising South Korea's efforts to essentially "flatten the curve" without any restrictive lockdown. We can take a look at the mobility trend compared with the the one plotted for the UK and see what can be learnt from their efforts.

In [30]:
Korea_walking = transpose_df('Republic of Korea', 'walking')
Korea_driving = transpose_df('Republic of Korea', 'driving')
sk_row = corona_data[(corona_data['Country/Region'] == 'Korea, South') & (corona_data['Province/State'].isna())]
SK_deaths = transpose_corona_row(sk_row)
SK_deaths.head()

,Country/Region,Value
Date,,
2020-01-22,"Korea, South",0
2020-01-23,"Korea, South",0
2020-01-24,"Korea, South",0
2020-01-25,"Korea, South",0
2020-01-26,"Korea, South",0


In [31]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=Korea_walking.index, y=Korea_walking.Value, mode='lines', name='Walking'), 
              secondary_y=False)
fig.add_trace(go.Scatter(x=Korea_driving.index, y=Korea_driving.Value, mode='lines', name='Driving'), 
              secondary_y=False)

fig.add_trace(go.Scatter(x=SK_deaths.index, y=SK_deaths.Value, mode='lines', name='Deaths'), secondary_y=True)

fig.update_layout(xaxis_title='Date', 
                  title='Mobility Trend data compared to cumulative COVID-19 deaths in South Korea')
fig.update_layout(shapes=[dict(type = 'line', y0 = 0, y1=0,
            x0 = 0, x1 = 0.94, xref='paper', 
            line=dict(color='black', width=2, dash='dot'))])
fig.add_annotation(x=pandas.Timestamp('2020-04-26'), y=5, xref='x', yref='y', text='Baseline Direction Requests', 
                   showarrow=False)
fig.update_yaxes(title_text='Percentage change in direction requests (%)', secondary_y=False)
fig.update_yaxes(title_text='Confirmed Cases (cumulative)', secondary_y=True)
fig.show(renderer='notebook_connected')

We can clearly see from the above plot that the mobility trends dropped significantly well before the first deaths were announced in the South Korea and continued to fall past the first deaths. This quick response by the South Koreans can not be stressed enough in terms of helping the country contain the number of deaths and cases and allowing the country to recover in a matter of weeks.

If the UK had taken a similar approach, we could see the number of deaths in the hundreds rather than the tens of thousands we are currently experiencing and that are still increasing.

While we continue to fight the coronavirus here in the UK, it is important for us to remember the lives that have been lost as well as the brave and strong NHS workers who continue to work long hours to take care of the sick. Without the support of the country, the task would be infinitely harder and I would like to express my condolences to those who have lost lives due to this virus. While this analysis reduces these deaths to just numbers, there is a life and story behind each one.